In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
train_df = pd.read_csv(r'E:\kaggle\houseprice\data\train.csv')
test_df = pd.read_csv(r'E:\kaggle\houseprice\data\test.csv')
sub_df = pd.read_csv(r'E:\kaggle\houseprice\data\sample_submission.csv')
test_df = pd.merge(test_df,sub_df,on='Id',)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 81 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

In [107]:
'''将数据合并'''
data = pd.concat([train_df,test_df],axis = 0)
'''先查看各项数据中NaN的含义'''
'''
1. Alley:NaN表示没有小路→替换为None；
2. BsmtQual:NaN表示没有地下室→替换为None；
3. BsmtCond:同上→替换为None；
4. BsmtExposure:同上→替换为None；
5. BsmtFinType1:同上→替换为None；
6. BsmtFinType2:同上→替换为None；
7. FireplaceQu:没有壁炉→替换为None；
8. GarageType:→替换为None；
9. GarageFinish:→替换为None；
10. GarageQual:→替换为None；
11. GarageCond:→替换为None；
12. PoolQC:→替换为None；
13. Fence:→替换为None；
14. MiscFeature:→替换为None。
'''
replace_list = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish',
     'GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
for a in replace_list:
    data[a] = data[a].fillna('None')

In [108]:
'''剩余数据'''
for k,v in data.isnull().sum().items():
    if v>0:
        print(k,v)

MSZoning 4
LotFrontage 486
Utilities 2
Exterior1st 1
Exterior2nd 1
MasVnrType 24
MasVnrArea 23
BsmtFinSF1 1
BsmtFinSF2 1
BsmtUnfSF 1
TotalBsmtSF 1
Electrical 1
BsmtFullBath 2
BsmtHalfBath 2
KitchenQual 1
Functional 2
GarageYrBlt 159
GarageCars 1
GarageArea 1
SaleType 1


In [109]:
'''LotFrontage以周边的中位数来处理'''
#data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median))
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].fillna(data['LotFrontage'].mean()))

In [110]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    data[col] = data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    data[col] = data[col].fillna(0)
data["MasVnrType"] = data["MasVnrType"].fillna("None")
data["MasVnrArea"] = data["MasVnrArea"].fillna(0)

In [111]:
for k,v in data.isnull().sum().items():
    if v>0:
        print(k,v)

MSZoning 4
Utilities 2
Exterior1st 1
Exterior2nd 1
Electrical 1
KitchenQual 1
Functional 2
SaleType 1


In [112]:
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])
data["Functional"] = data["Functional"].fillna("Typ")
data = data.drop(['Utilities'], axis=1)
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])

In [113]:
for k,v in data.isnull().sum().items():
    if v>0:
        print(k,v)
'''全部完成'''

'全部完成'

In [114]:
#object对象分成多维处理
dfobj = data.select_dtypes(include=['object'])
fobj = data.select_dtypes(exclude=['object'])
odf = pd.get_dummies(dfobj)
data = pd.concat([fobj,odf],axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Columns: 302 entries, Id to SaleCondition_Partial
dtypes: float64(12), int64(26), uint8(264)
memory usage: 1.6 MB


In [115]:
#剩余所有元素进行归一化处理
data_norm = (data-data.min())/(data.max()-data.min())
data_norm

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.000000,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.122500,0.125089,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000343,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.000000,0.173281,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000685,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.101250,0.086109,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.001028,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.000000,0.038271,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.001371,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.218750,0.116052,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.001714,0.176471,0.219178,0.059899,0.444444,0.500,0.876812,0.750000,0.000000,0.129695,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.002056,0.000000,0.184932,0.041057,0.777778,0.500,0.956522,0.916667,0.116250,0.242558,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.002399,0.235294,0.165431,0.042450,0.666667,0.625,0.731884,0.383333,0.150000,0.152197,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.002742,0.176471,0.102740,0.022529,0.666667,0.500,0.427536,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.003084,1.000000,0.099315,0.028605,0.444444,0.625,0.485507,0.000000,0.000000,0.150780,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [116]:
'''开始建模'''
'''数值数据直接使用，字符型数据使用pd.get_dummies()'''
train = data_norm[:1460]
test = data_norm[1460:]
train.shape,test.shape#重新将train和test分开，test中的price列是无用的，需要删除

((1460, 302), (1459, 302))

In [117]:
train.to_csv('E:\kaggle\houseprice\data\Mtrain.csv',index = False,sep = ',')
test.to_csv('E:\kaggle\houseprice\data\Mtest.csv',index = False,sep=',')